In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

In [3]:
#　会議録読込み
loader = PyPDFLoader("000.pdf")

In [ ]:
# PDF ドキュメントからテキストを抽出し、抽出したテキストをページごとに分割
pages = loader.load_and_split()

In [ ]:
# OpenAI の API Key 入力　
os.environ["OPENAI_API_KEY"] = '000'
openai.api_key = os.getenv("OPENAI_API_KEY")
# モデル選択
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
# 分割したテキストについて、テキスト間の関連性に関するベクターデータをベクターストアに格納
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

In [ ]:
# 回答文の作成に関連した元テキスト群についても示すように指定

pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

query = """
以上のWeb会議の会話履歴から、000さんについて以下の項目をまとめてなさい。
①今の仕事について（職場でのトラブル、仕事へのモチベーションなど）
②今後のキャリアについて（今の仕事の契約更新の意向、転職の意向など）
③体調について（怪我、病気など）
④精神状態について（ストレス、悩み、人間関係など）
⑤教育研修について（社内でのOJT、レクチャー、業務の引継ぎ、研修、講習など）
⑥採用について（採用の計画、増員、他派遣スタッフの退職情報、他派遣スタッフの入社情報など）
⑦その他の会話内容について

#制約条件
・会話履歴が日本語以外の言語であっても、日本語でまとめなさい。
・話し合われた内容とその結論についてできる限り詳しくまとめなさい。
・それぞれの項目について答えが複数ある場合は、箇条書きで全て記載しなさい。

#出力形式
Ｗeb会議のタイトル（1行目に記載されているので、そのまま引用しなさい）

①今の仕事について


②今後のキャリアについて


③体調について


④精神状態について


⑤教育研修について


⑥採用について


⑦その他の会話内容について

"""
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

In [ ]:
print(result["answer"])